## 01. Data Preparation

In [2]:
# Библиотеки
import import_ipynb
from config import RAW_JSON_PATH, ERRORS_JSON_PATH, PROCESSED_CSV_PATH, CENTROIDS_CSV_PATH, CENTROID_ERRORS_CSV_PATH
from config import REGION_CODE_LENGTH
from config import MIN_POLYGON_POINTS, ROUND_AREA_SC63, ROUND_AREA_WGS84, ROUND_ASPECT_RATIO
#from 00_utils import root_mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import re
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Polygon, MultiPolygon, Point

from IPython.display import HTML
from IPython.display import display

#### 🔹 1. Загрузка исходного JSON

In [6]:
# Загрузка исходного JSON
def load_json(path):  
    with open(path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    return raw_data

In [7]:
# "аномизация" кадастровых номеров
# выполняем один раз перед публикацией на github
raw_json = load_json(RAW_JSON_PATH)
for idx, item in enumerate(raw_json):
    item["number"] = f"sample_{idx:06_}"

# Сохранение в новый файл
ANONYMIZED_PATH = "mnt/data/numbers_polygons_anonymized.json"

with open(ANONYMIZED_PATH, "w", encoding="utf-8") as f:
    json.dump(raw_json, f, ensure_ascii=False, indent=2)

print(f"✅ Saved: {ANONYMIZED_PATH}")

✅ Saved: mnt/data/numbers_polygons_anonymized.json


In [4]:
help(json.load)

Help on function load in module json:

load(fp, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)
    Deserialize ``fp`` (a ``.read()``-supporting file-like object containing
    a JSON document) to a Python object.
    
    ``object_hook`` is an optional function that will be called with the
    result of any object literal decode (a ``dict``). The return value of
    ``object_hook`` will be used instead of the ``dict``. This feature
    can be used to implement custom decoders (e.g. JSON-RPC class hinting).
    
    ``object_pairs_hook`` is an optional function that will be called with the
    result of any object literal decoded with an ordered list of pairs.  The
    return value of ``object_pairs_hook`` will be used instead of the ``dict``.
    This feature can be used to implement custom decoders.  If ``object_hook``
    is also defined, the ``object_pairs_hook`` takes priority.
    
    To use a custom ``JSONDecod

#### 🔹 2. Парсинг и извлечение координат

In [6]:
## Парсинг полигонов из JSON
def parse_multipolygon_coords(coords):
    polygons = []
    for poly_coords in coords:
        if len(poly_coords) > 0:
            polygons.append(poly_coords[0])  # только внешний контур
    return polygons

## Функция обработки JSON
def processing_raw_json(raw_json):
    
    records = []
    errors = []

    for idx, item in enumerate(raw_json):
        if not isinstance(item, dict):
            print(f"❌ Missing element #{idx}: incorrect record — {item}")
            continue
        number = item.get("number", f"no_number_{idx}")
        try:
            # --- SC63 --- #
            sc63_raw = item.get("loof_polygon")
            if not sc63_raw:
                raise ValueError("loof_polygon is missing or null")
            sc63_json = json.loads(sc63_raw)
            coords_sc63 = sc63_json.get("coordinates")
            if not coords_sc63:
                raise ValueError("SC63 coordinates missing")
            sc63_polygons = parse_multipolygon_coords(coords_sc63)

            # --- WGS84 --- #
            cadastral_raw = item.get("cadastr.live polygon")
            if not cadastral_raw:
                raise ValueError("cadastr.live polygon is missing or null")

            feature_collection = json.loads(cadastral_raw)
            features = feature_collection.get("features")
            if not features or not isinstance(features[0], dict):
                raise ValueError("features array is missing or not valid")

            geometry_raw = features[0].get("geometry")
            if not geometry_raw or geometry_raw == "null":
                raise ValueError("geometry is missing or invalid")

            if isinstance(geometry_raw, str):
                wgs84_geom = json.loads(geometry_raw)
            else:
                wgs84_geom = geometry_raw
            coords_wgs = wgs84_geom.get("coordinates")
            if not coords_wgs:
                raise ValueError("WGS84 coordinates missing")

            wgs84_polygons = parse_multipolygon_coords(coords_wgs)

            if len(sc63_polygons) != len(wgs84_polygons):
                raise ValueError("Polygon count mismatch")
            for sc63_poly, wgs84_poly in zip(sc63_polygons, wgs84_polygons):
                for i, ((sc_x, sc_y), (lon, lat)) in enumerate(zip(sc63_poly, wgs84_poly)):
                    records.append({
                        "number": number,
                        "vertex_id": i,  # ← индекс вершины внутри полигона
                        "sc63_x": sc_x,
                        "sc63_y": sc_y,
                        "wgs84_lon": lon,
                        "wgs84_lat": lat
                    })
        except Exception as e:
            print(f"❌ Error in {number} (#{idx}): {e}")
            errors.append({"number": number, "error": str(e)})

    return records, errors


raw_data = load_json(RAW_JSON_PATH)
records, errors = processing_raw_json(raw_data)
pd.DataFrame(errors).to_csv(ERRORS_JSON_PATH, index=False, encoding="utf-8")
df_points = pd.DataFrame(records)

❌ Error in 4825785000:01:000:0039 (#89): geometry is missing or invalid
❌ Error in 4825785000:01:000:0027 (#419): geometry is missing or invalid
❌ Error in 4623083200:18:000:0110 (#1323): geometry is missing or invalid
❌ Error in 6523285500:02:001:0123 (#1604): geometry is missing or invalid
❌ Error in 4623083200:13:000:0637 (#1733): geometry is missing or invalid
❌ Error in 6523285500:02:001:0117 (#1742): geometry is missing or invalid
❌ Error in 4623086800:01:000:0083 (#1756): geometry is missing or invalid
❌ Error in 6523285500:02:001:0148 (#1882): geometry is missing or invalid
❌ Error in 1825082600:03:000:7169 (#2059): geometry is missing or invalid
❌ Error in 4621287400:01:001:0025 (#2252): geometry is missing or invalid
❌ Error in 4623083200:18:000:0108 (#2823): geometry is missing or invalid
❌ Error in 4621286800:03:002:0031 (#3062): geometry is missing or invalid
❌ Error in 4621286800:03:002:0030 (#3087): geometry is missing or invalid
❌ Error in 1825087400:08:000:7224 (#3283)

In [7]:
df_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57563 entries, 0 to 57562
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     57563 non-null  object 
 1   vertex_id  57563 non-null  int64  
 2   sc63_x     57563 non-null  float64
 3   sc63_y     57563 non-null  float64
 4   wgs84_lon  57563 non-null  float64
 5   wgs84_lat  57563 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.6+ MB


In [8]:
df_points.head(5)

,number,vertex_id,sc63_x,sc63_y,wgs84_lon,wgs84_lat
0,4625382600:02:000:0270,0,1317984.10,5453536.23,23.745505,49.296396
1,4625382600:02:000:0270,1,1317871.27,5453327.00,23.743946,49.294518
2,4625382600:02:000:0270,2,1317895.32,5453315.03,23.744274,49.294410
3,4625382600:02:000:0270,3,1318007.09,5453522.29,23.745819,49.296270
4,4625382600:02:000:0270,4,1317984.10,5453536.23,23.745505,49.296396


#### 🔹 3. Валидация и фильтрация (min 3 точки, валидность)

In [6]:
def validate_geometry(df, min_points=MIN_POLYGON_POINTS):
    valid_counts = df.groupby("number").size()
    valid_numbers = valid_counts[valid_counts >= min_points].index
    print(f"❌ Invalid polygons: {len(valid_counts) - len(valid_numbers)}")
    display(valid_counts[valid_counts < min_points])    
    return df[df["number"].isin(valid_numbers)].copy()

df_points = validate_geometry(df_points, min_points=MIN_POLYGON_POINTS)

❌ Invalid polygons: 0


Series([], dtype: int64)

#### 🔹 4. Feature engineering:

In [7]:
# - `region_code`
def extract_region_code(cadastr_number):
    match = re.match(r"^(\d+):", cadastr_number)
    if match:
        full = match.group(1)
        return full[:REGION_CODE_LENGTH]  # обрезаем до нужной длины
    return None

# - `area_in_m2_from_wgs` - площадь в м² из WGS84
def area_in_m2_from_wgs(lon_lat_coords):
    try:
        poly = Polygon(lon_lat_coords)
        gdf = gpd.GeoSeries([poly], crs="EPSG:4326")
        gdf_proj = gdf.to_crs(epsg=3857)  # Web Mercator в метрах
        return gdf_proj.area.values[0]
    except Exception as e:
        print(f"❌ Error in area calculation: {e}")
        return np.nan

from shapely.validation import explain_validity

# - `centroid` `areas` `aspect ratio`
def compute_centroids_and_areas(df_points, fix_invalid=True, log_errors=True):
    results = []
    error_log = []

    for number, group in df_points.groupby("number"):
        row = {"number": number}
        error_entry = {"number": number, "wgs84_error": None, "sc63_error": None}

        # --- WGS84 --- #
        poly = None
        coords_wgs = list(zip(group["wgs84_lon"], group["wgs84_lat"]))
        if len(coords_wgs) >= MIN_POLYGON_POINTS:
            try:
                poly_wgs = Polygon(coords_wgs)
                if poly_wgs is None:
                    print(f"⚠️ {number} → poly WGS is None after Polygon creation")
                elif not poly_wgs.is_valid:
                    print(f"⚠️ {number} → Invalid polygon WGS: {explain_validity(poly_wgs)}")
                    if fix_invalid:
                        print(f"🔧 Attempting to fix polygon WGS via buffer(0)...")
                        poly_wgs = poly_wgs.buffer(0)
                        if poly_wgs.is_valid:
                            print(f"✅ {number} → Polygon WGS fixed successfully")
                        else:
                            print(f"❌ {number} → Polygon WGS still invalid after fix")

                if poly_wgs.is_valid:                    
                    row["wgs84_centroid_lon"] = round(poly_wgs.centroid.x, 6)
                    row["wgs84_centroid_lat"] = round(poly_wgs.centroid.y, 6)
                    row["polygon_area_wgs84"] = round(area_in_m2_from_wgs(coords_wgs), ROUND_AREA_WGS84)
                else:
                    error_entry["wgs84_error"] = explain_validity(poly)
            except Exception as e:
                print(f"❌ Error in WGS84 for {number}: {e}")

        # --- SC63 --- #
        poly = None
        coords_sc = list(zip(group["sc63_x"], group["sc63_y"]))
        if len(coords_sc) >= MIN_POLYGON_POINTS:
            try:
                poly_sc = Polygon(coords_sc)
                if poly_sc is None:
                    print(f"⚠️ {number} → poly SC is None after Polygon creation")
                elif not poly_sc.is_valid:
                    print(f"⚠️ {number} → Invalid polygon SC: {explain_validity(poly_sc)}")
                    if fix_invalid:
                        print(f"🔧 Attempting to fix polygon SC via buffer(0)...")
                        poly_sc = poly_sc.buffer(0)
                        if poly_sc.is_valid:
                            print(f"✅ {number} → Polygon SC fixed successfully")
                        else:
                            print(f"❌ {number} → Polygon SC still invalid after fix")

                if poly_sc.is_valid:
                    row["sc63_centroid_x"] = round(poly_sc.centroid.x, 2)
                    row["sc63_centroid_y"] = round(poly_sc.centroid.y, 2)
                    row["polygon_area_sc63"] = round(poly_sc.area, ROUND_AREA_SC63)
                else:
                    error_entry["sc63_error"] = explain_validity(poly_sc)                    
            except Exception as e:
                print(f"❌ Error in SC63 for {number}: {e}")

        # --- Area Ratio --- #
        if row.get("polygon_area_wgs84") and row.get("polygon_area_sc63"):
            try:
                row["area_ratio"] = round(row["polygon_area_wgs84"] / row["polygon_area_sc63"], ROUND_ASPECT_RATIO)
            except ZeroDivisionError:
                row["area_ratio"] = np.nan

        results.append(row)
        if log_errors and (error_entry["wgs84_error"] or error_entry["sc63_error"]):
            error_log.append(error_entry)        

    return pd.DataFrame(results), pd.DataFrame(error_log)

# - может быть: bbox, aspect ratio


# Применяем функции к DataFrame
df_points["region_code"] = df_points["number"].apply(extract_region_code)

df_centroids, df_centroid_errors = compute_centroids_and_areas(df_points)
df_centroids.to_csv(CENTROIDS_CSV_PATH, index=False)
df_centroid_errors.to_csv(CENTROID_ERRORS_CSV_PATH, index=False)

cols_to_remove = [c for c in df_points.columns if "centroid" in c or "area" in c]
df_points.drop(columns=cols_to_remove, inplace=True)
df_points = df_points.merge(df_centroids, on="number", how="left")

# - `vertex_id_norm` - нормализованный индекс вершины
df_points["vertex_id_norm"] = df_points.groupby("number")["vertex_id"].transform(
    lambda x: x / x.max()
)


⚠️ 1821784000:01:000:0047 → Invalid polygon WGS: Ring Self-intersection[27.9813083746895 50.772824721293]
🔧 Attempting to fix polygon WGS via buffer(0)...
✅ 1821784000:01:000:0047 → Polygon WGS fixed successfully
⚠️ 1821784000:01:000:0047 → Invalid polygon SC: Ring Self-intersection[3192997.5 5618818.83]
🔧 Attempting to fix polygon SC via buffer(0)...
✅ 1821784000:01:000:0047 → Polygon SC fixed successfully
⚠️ 4623388000:07:000:0014 → Invalid polygon WGS: Ring Self-intersection[24.5128142480228 49.6667029551743]
🔧 Attempting to fix polygon WGS via buffer(0)...
✅ 4623388000:07:000:0014 → Polygon WGS fixed successfully
⚠️ 4823383900:01:000:0077 → Invalid polygon SC: Self-intersection[4272348.01845865 5215377.76697805]
🔧 Attempting to fix polygon SC via buffer(0)...
✅ 4823383900:01:000:0077 → Polygon SC fixed successfully
⚠️ 5924187900:04:002:0261 → Invalid polygon WGS: Self-intersection[33.7046677363908 50.7723927397934]
🔧 Attempting to fix polygon WGS via buffer(0)...
✅ 5924187900:04:00

In [8]:
df_points.head(5)

,number,vertex_id,sc63_x,sc63_y,wgs84_lon,wgs84_lat,region_code,wgs84_centroid_lon,wgs84_centroid_lat,polygon_area_wgs84,sc63_centroid_x,sc63_centroid_y,polygon_area_sc63,area_ratio,vertex_id_norm
0,4625382600:02:000:0270,0,1317984.10,5453536.23,23.745505,49.296396,46253,23.744886,49.295399,14859.25,1317939.43,5453425.15,6352.77,2.339019,0.00
1,4625382600:02:000:0270,1,1317871.27,5453327.00,23.743946,49.294518,46253,23.744886,49.295399,14859.25,1317939.43,5453425.15,6352.77,2.339019,0.25
2,4625382600:02:000:0270,2,1317895.32,5453315.03,23.744274,49.294410,46253,23.744886,49.295399,14859.25,1317939.43,5453425.15,6352.77,2.339019,0.50
3,4625382600:02:000:0270,3,1318007.09,5453522.29,23.745819,49.296270,46253,23.744886,49.295399,14859.25,1317939.43,5453425.15,6352.77,2.339019,0.75
4,4625382600:02:000:0270,4,1317984.10,5453536.23,23.745505,49.296396,46253,23.744886,49.295399,14859.25,1317939.43,5453425.15,6352.77,2.339019,1.00


In [9]:
# функция конвертации SC63 в WGS84

# sc63_to_wgs84_transform.py
from math import cos, sin, tan, sqrt, atan, atan2, radians, degrees
import mpmath
from mpmath import mpf

def tcoord(E,N):
    # help function
    Z = 0.0
    N = mpf(N)
    E = mpf(E)
    sin = mpmath.sin
    cos = mpmath.cos
    tan = mpmath.tan
    atan = mpmath.atan
    lat = 0
    lon = 0
    B = mpmath.radians(lat)
    L = mpmath.radians(lon)
    def decdeg2dms(dd):
        dd = float(dd)
        is_positive = dd >= 0
        dd = abs(dd)
        minutes,seconds = divmod(dd*3600,60)
        degrees,minutes = divmod(minutes,60)
        degrees = degrees if is_positive else -degrees
        seconds = round(seconds,4)
        return (degrees,minutes,seconds)


    # constant
    a = mpf(6378245.000) # meter # велика піввісь еліпса
    f = mpf(1.0)/298.3 # стиснення еліпса
    b = a - a*f
    e = mpmath.sqrt((a**2-b**2)/a**2)
    e_2 = mpmath.sqrt((a**2-b**2)/b**2) # e'
    B0 = 0                  # fi0
    L0 = mpmath.radians(29.5) # lambda0
    FN = mpf(-9214.6880) #meter
    FE = 0.0 #meter

    # Перевірка зони
    zone = int(E/1000000)
    l = 0

    #=IF(D5=1,RADIANS(23.5),IF(D5=2,RADIANS(26.5),IF(D5=3,RADIANS(29.5),IF(D5=4,RADIANS(32.5),IF(D5=5,RADIANS(35.5),IF(D5=6,RADIANS(38.5),""))))))
    try:
        if zone == 1:
            l = 23.5
        elif zone == 2:
            l = 26.5
        elif zone == 3:
            l = 29.5
        elif zone == 4:
            l = 32.5
        elif zone == 5:
            l = 35.5
        elif zone == 6:
            l = 38.5
    except NameError:
       print( NameError, u"error: Координати не містять номера зони СК64, за потреби зверніться до Автора Програми: pavlo.dazru@gmail.com")

    #=IF(D5=1,1300000,IF(D5=2,2300000,IF(D5=3,3300000,IF(D5=4,4300000,IF(D5=5,5300000,IF(D5=6,6300000,""))))))
    try:
        if zone == 1:
            FE = 1300000.0
        elif zone == 2:
            FE = 2300000.0
        elif zone == 3:
            FE = 3300000.0
        elif zone == 4:
            FE = 4300000.0
        elif zone == 5:
            FE = 5300000.0
        elif zone == 6:
            FE = 6300000.0
    except NameError:
       print( NameError, u"error: Координати не містять номера зони СК64, за потреби зверніться до Автора Програми: pavlo.dazru@gmail.com")
    FN = mpf(FN)
    FE = mpf(FE)
    l  = mpf(l)


    L0 = mpmath.radians(l)
    k0 = 1.0
    M1 = N - FN
    m1 = M1 / (a*(1-(e**2/4)-(3*(e**4/64))-(5*(e**6/256))))
    e1 = (1-mpmath.sqrt(1-e**2))/(1+mpmath.sqrt(1-e**2))
    B1 = m1 + ((3*e1/2)-(27*e1**3/32))*sin(2*m1)+((21*e1**2/16)-(55*e1**4/32))*sin(4*m1)+(151*(e1**3/96)*sin(6*m1))+((1097*e1**4/512)*sin(8*m1))
    T1 = tan(B1)**2
    C1 = e_2**2*(cos(B1)**2)
    V1 = a / (mpmath.sqrt(1-e**2*(sin(B1)**2)))
    p1 = (a*(1-e**2))/(1-e**2*(sin(B1)**2))**1.5
    D = (E-FE)/(V1*k0)
    Bsk42_rad = B1-((V1*tan(B1))/p1)*(((D**2)/2)-(((5+3*T1+10*C1-4*C1**2-9*e_2**2)*D**4)/24)+(((61+90*T1+298*C1+45*T1**2-252*e_2**2)*D**6)/720))
    Lsk42_rad = L0 + ((D-((1+2*T1+C1)*D**3)/6)+((5-2*C1+28*T1-3*C1**2+8*e_2**2+24*T1**2)*(D**5)/120))/cos(B1)
    #=D13+(D27-(1+2*D23+D24)*POWER(D27,3)/6+(5-2*D24+28*D23-3*POWER(D24,2)+8*POWER(D11,2)+24*POWER(D23,2))*POWER(D27,5)/120)/COS(D22)
    Bsk42_ddeg = mpmath.degrees(Bsk42_rad)
    Lsk42_ddeg = mpmath.degrees(Lsk42_rad)

    # Перевірка заокруглень до еселя

    # print( "L0", round(L0,15)-float(0.51487212933832700000), L0
    # print( "D ", round(D,16)  - mpf(0.01110154954724890000), D
    # print( "T1", round(T1,14)  -mpf(1.47205393116797000000), T1
    # print( "C1", round(C1,16)  -mpf(0.00272588123168480000), C1
    # print( "e_2", round(e_2,16) -mpf(0.08208852182055300000), e_2
    # print( "cos(B1)", round(cos(B1),15) -mpf(0.63602037621171300000), cos(B1)
    # print( Bsk42_rad, '\t', Lsk42_rad
    # print( round(Bsk42_rad,15)-0.88139127223893300000,'\t\t\t\t', round(Lsk42_rad,15)-0.53232542185590200000
    # print( Bsk42_ddeg, Lsk42_ddeg
    # print( round(Bsk42_ddeg,15)-50.49999999895700000000, round(Lsk42_ddeg,13)-30.49999999986430000000
    # print( "Lsk42", decdeg2dms(Lsk42_ddeg)
    # print( "Bsk42", decdeg2dms(Bsk42_ddeg)
    #print( Btemp[0],Btemp[1], Btemp[2]


    #  # # # # # # # # # включити заокруглення
    # Bg = int(Bsk42_ddeg); Bminut = round((Bsk42_ddeg-Bg)*60,0); Bsec = round((60*(Bsk42_ddeg-Bg)-Bminut)*60,4)
    # Lg = int(Lsk42_ddeg); Lminut = round((Lsk42_ddeg-Lg)*60,0); Lsec = round((60*(Lsk42_ddeg-Lg)-Lminut)*60,4)
    # Bround = Bg + Bminut/60 + Bsec/3600
    # Lround = Lg + Lminut/60 + Lsec/3600
    # B = mpmath.radians(Bround)
    # L = mpmath.radians(Lround)


    #print( Bg, Bminut, Bsec
    #print( Lg, Lminut, Lsec
    #print( Bround
    #print( Lround

    # # Якшо не коментоване заокрулгення вимкнуте
    B = Bsk42_rad
    L = Lsk42_rad


    # Крок 2 конвертація B,L до X,Y,Z
     #=D9/SQRT((1-POWER(D12,2)*POWER(SIN(D4),2)))
    V = a / (mpmath.sqrt(1-(e**2*(sin(B)**2))))
    #print( "V" , round(V,8)-6390992.7038410900, V

    X = V*cos(B)*cos(L)
    Y = V*cos(B)*sin(L)
    Z = ((1-e**2)*V)*sin(B)

    # Перевірка заокруглень
    # print( "type x", type(X), type(Y)
    # print( "x", round(X,8)-3502670.1039987200, X
    # print( "y", round(Y,8)-2063230.3689257000, Y
    # print( "z", round(Z,8)-4898438.8280005800, Z


    # Крок 3 застосування трансформації Гельмерта

    # Параметри трансформації
    tX = 30.918
    tY = -119.346
    tZ = -93.514
    rX = mpmath.radians(-0.636831/3600)
    rY = mpmath.radians(0.242067/3600)
    rZ = mpmath.radians(-0.56995/3600)
    m = 0.00000009

    XYZsk42 = np.matrix([X,Y,Z])
    Helm = np.matrix([[m,-rZ,+rY],[+rZ,m,-rX],[-rY,+rX,m]])
    dDelta = np.matrix([tX,tY,tZ])
    XYZwgs = XYZsk42 + XYZsk42*Helm + dDelta
    Xwgs = XYZwgs.item((0,0))
    Ywgs = XYZwgs.item((0,1))
    Zwgs = XYZwgs.item((0,2))

    # Перевірка заокруглень
    # print( "Xwgs", round(Xwgs,8)-3502689.88744988000, Xwgs
    # print( "Ywgs", round(Ywgs,8)-2063105.76352431000, Ywgs
    # print( "Zwgs", round(Zwgs,8)-4898356.23561025000, Zwgs



    # Крок 4 Конвертація X,Y,Z до B,L
    aWGS = 6378137.0000
    fWGS = 1/298.2572236
    bWGS = aWGS - aWGS*fWGS
    eWGS = mpmath.sqrt((aWGS**2-bWGS**2)/aWGS**2)
    eWGS_2 = mpmath.sqrt((aWGS**2-bWGS**2)/bWGS**2)
    ePSL = eWGS**2/(1-eWGS**2)
    p = mpmath.sqrt(Xwgs**2+Ywgs**2)
    q = atan((Zwgs*aWGS)/(p*bWGS))
    Bwgs_rad = atan((Zwgs+ePSL*bWGS*(sin(q))**3)/(p-(eWGS**2)*aWGS*(cos(q)**3)))
    Bwgs = mpmath.degrees(Bwgs_rad)
    Lwgs_rad = atan(Ywgs/Xwgs)
    Lwgs = mpmath.degrees(Lwgs_rad)

    # print( N, E, Bwgs, Lwgs
    # print( decdeg2dms(Bwgs), decdeg2dms(Lwgs)
    return ([round(Lwgs,13),round(Bwgs,13),0.0,zone])

def sc63_to_wgs84(x, y):
    lon, lat, _, zone = tcoord(x, y)
    return lat, lon, zone


df_points[["wgs84_baseline_lat", "wgs84_baseline_lon", "zone"]] = df_points.apply(
    lambda row: sc63_to_wgs84(row["sc63_x"], row["sc63_y"]), axis=1, result_type="expand"
)

df_points["delta_lon_from_baseline"] = df_points["wgs84_baseline_lon"] - df_points["wgs84_lon"]
df_points["delta_lat_from_baseline"] = df_points["wgs84_baseline_lat"] - df_points["wgs84_lat"]

In [14]:
display(HTML(df_points[["number","sc63_x", "sc63_y", "wgs84_baseline_lat", "wgs84_baseline_lon", "wgs84_lon", "wgs84_lat" , "delta_lon_from_baseline", "delta_lat_from_baseline", "zone"]].head(10).to_html(index=False)))

number,sc63_x,sc63_y,wgs84_baseline_lat,wgs84_baseline_lon,wgs84_lon,wgs84_lat,delta_lon_from_baseline,delta_lat_from_baseline,zone
4625382600:02:000:0270,1317984.10,5453536.23,49.296452,23.745577,23.745505,49.296396,0.000072,0.000056,1.0
4625382600:02:000:0270,1317871.27,5453327.00,49.294574,23.744016,23.743946,49.294518,0.000070,0.000056,1.0
4625382600:02:000:0270,1317895.32,5453315.03,49.294466,23.744346,23.744274,49.294410,0.000072,0.000056,1.0
4625382600:02:000:0270,1318007.09,5453522.29,49.296326,23.745892,23.745819,49.296270,0.000073,0.000056,1.0
4625382600:02:000:0270,1317984.10,5453536.23,49.296452,23.745577,23.745505,49.296396,0.000072,0.000056,1.0
4625382600:02:000:0257,1318204.17,5453161.37,49.293075,23.748585,23.748514,49.293018,0.000071,0.000057,1.0
4625382600:02:000:0257,1318236.35,5453145.36,49.292930,23.749027,23.748954,49.292875,0.000073,0.000055,1.0
4625382600:02:000:0257,1318324.47,5453329.86,49.294586,23.750247,23.750176,49.294530,0.000071,0.000057,1.0
4625382600:02:000:0257,1318302.29,5453343.31,49.294708,23.749942,23.749871,49.294653,0.000072,0.000055,1.0
4625382600:02:000:0257,1318204.17,5453161.37,49.293075,23.748585,23.748514,49.293018,0.000071,0.000057,1.0


In [19]:
# Дорабатываем центроиды: пересчёт координат в WGS84, расчёт сдвига, зона
df_centroids[["wgs84_centroid_baseline_lat", "wgs84_centroid_baseline_lon", "zone"]] = df_centroids.apply(
    lambda row: sc63_to_wgs84(row["sc63_centroid_x"], row["sc63_centroid_y"]), axis=1, result_type="expand"
)

df_centroids["delta_lon_from_baseline"] = df_centroids["wgs84_centroid_baseline_lon"] - df_centroids["wgs84_centroid_lon"]
df_centroids["delta_lat_from_baseline"] = df_centroids["wgs84_centroid_baseline_lat"] - df_centroids["wgs84_centroid_lat"]

display(HTML(df_centroids[["number","sc63_centroid_x", "sc63_centroid_y", "wgs84_centroid_baseline_lat", "wgs84_centroid_baseline_lon", "wgs84_centroid_lon", "wgs84_centroid_lat" , "delta_lon_from_baseline", "delta_lat_from_baseline", "zone"]].head(10).to_html(index=False)))

number,sc63_centroid_x,sc63_centroid_y,wgs84_centroid_baseline_lat,wgs84_centroid_baseline_lon,wgs84_centroid_lon,wgs84_centroid_lat,delta_lon_from_baseline,delta_lat_from_baseline,zone
0520281400:01:004:0012,2370539.90,5413153.67,48.929649,27.461045,27.460983,48.929602,0.000062,0.000047,2.0
0520281400:01:004:0071,2372004.17,5412262.49,48.921468,27.480869,27.480806,48.921421,0.000063,0.000047,2.0
0520281400:01:005:0003,2371607.49,5411996.38,48.919121,27.475410,27.475348,48.919075,0.000062,0.000046,2.0
0520281400:02:003:0127,2368157.52,5410102.67,48.902484,27.428028,27.427966,48.902435,0.000062,0.000049,2.0
0520282600:04:016:0020,2392346.74,5401586.58,48.822798,27.755978,27.755919,48.822751,0.000059,0.000047,2.0
0520282600:04:016:0030,2392088.67,5401510.73,48.822154,27.752447,27.752389,48.822108,0.000058,0.000046,2.0
0520282600:04:016:0031,2392584.89,5401464.77,48.821667,27.759193,27.759134,48.821621,0.000059,0.000046,2.0
0520282600:04:016:0041,2392303.15,5402088.39,48.827316,27.755497,27.755437,48.827270,0.000060,0.000046,2.0
0520282600:04:016:0047,2392740.35,5401559.70,48.822498,27.761330,27.761273,48.822452,0.000057,0.000046,2.0
0520481300:03:001:0318,3313319.33,5333272.69,48.215220,29.677650,29.677593,48.215175,0.000057,0.000045,3.0


In [10]:
## 🔹 5. Сохранение в points_processed.csv
df_points.to_csv(PROCESSED_CSV_PATH, index=False, encoding="utf-8")
df_centroids.to_csv(CENTROIDS_CSV_PATH, index=False, encoding="utf-8")

NameError: name 'df_centroids' is not defined